This notebook shows the functionalities of several LICP related methods. We use two different approaches
1. loli
2. pirca

Loli is the code relating to the published paper https://proceedings.mlr.press/v244/mey24a.html .
pirca is the code relating to an unpublished paper and has several relaxations over loli, targeting the root cause identification case

In [32]:
n=100
import numpy as np
A=np.random.pareto(3,size=(n,1))
B=np.random.pareto(3,size=(n,1))
print(min([np.var(A),np.var(B)])/max([np.var(A),np.var(B)]))

0.5406270879677607


## We start with loli: loli makes a normal noise assumption, hass little robustness against deviations from that, and has no robustness against the homogeneity assumption.

In [18]:
import numpy as np
import loli
from sklearn.linear_model import LinearRegression

### The first experiment shows that loli works under the gaussian assumptiom

In [2]:
# We have a simple example with three variables for which we have each 1000 samples
np.random.seed(42)
n=1000
X=np.zeros((n,3))

X_loli=[]
Y_loli=[]

# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))
X_loli.append(np.concatenate([X_normal['0'],X_normal['1']],axis=1)) # Data from each environment is appended to a list
Y_loli.append(X_normal['Y'])
    
# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,1,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))
X_loli.append(np.concatenate([X_faulty['0'],X_faulty['1']],axis=1))
Y_loli.append(X_faulty['Y'])


B=1000
plausibleS=loli.gauss(X_loli,Y_loli,B=B)
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}
print('Estimated parents of variable Y  under normal noise:', supphat)

Estimated parents of variable Y  under normal noise: {0}


### If we change the target noise to a heavy tailed distribution loli fails

In [3]:
# We have a simple example with three variables for which we have each 1000 samples
np.random.seed(41)
n=1000
X=np.zeros((n,3))

X_loli=[]
Y_loli=[]

# We have the simple causal structure X0->X1->X2, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.pareto(3,size=(n,1)) # THIS IS NOT NORMAL HERE
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 
X_loli.append(np.concatenate([X_normal['0'],X_normal['1']],axis=1)) # Data from each environment is appended to a list
Y_loli.append(X_normal['Y'])
    
# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.pareto(3,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 
X_loli.append(np.concatenate([X_faulty['0'],X_faulty['1']],axis=1))
Y_loli.append(X_faulty['Y'])


B=1000
plausibleS=loli.gauss(X_loli,Y_loli,B=B)
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}
print('Estimated parents of variable Y  under heavy tailed noise:', supphat)
print('Note: depending on the random seed we might get different wrong answers, or the correct answer by chance')

Estimated parents of variable Y  under heavy tailed noise: {}
Note: depending on the random seed we might get different wrong answers, or the correct answer by chance


### If the homogeneous noise assumption is violated, loli fails

In [4]:
# We have a simple example with three variables for which we have each 1000 samples
np.random.seed(42)
n=1000
X=np.zeros((n,3))

X_loli=[]
Y_loli=[]

# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))  # TARGET NOISE IS NOT HOMOGENOUS
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))
X_loli.append(np.concatenate([X_normal['0'],X_normal['1']],axis=1)) # Data from each environment is appended to a list
Y_loli.append(X_normal['Y'])
    
# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,2,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))
X_loli.append(np.concatenate([X_faulty['0'],X_faulty['1']],axis=1))
Y_loli.append(X_faulty['Y'])


B=1000
plausibleS=loli.gauss(X_loli,Y_loli,B=B)
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}
print('Estimated parents of variable Y  under homogeneity violation:', supphat)

Estimated parents of variable Y  under homogeneity violation: {}


## Now we test if the several relations of pirca can deal with the violations of above

In [5]:
from LICP.licp import pirca
from sklearn.linear_model import LinearRegression

#### pirca has several relaxations over loli. The first relaxation is the removal of the gaussian noise assumption. Instead pirca uses a permutation test. So we expect that this setting has (some) robustness against heavy tails, but not against homegeneity violation

In [17]:
# We have a simple example with three variables for which we have each 1000 samples
np.random.seed(42)
n=1000
X=np.zeros((n,3))

model=LinearRegression()
pirca_inst=pirca(model=model)

# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))

# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,1,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))

data=[X_normal,X_faulty]

pirca_inst.compute_parents(data,targets=['Y'],test='permutation')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}
print('Estimated parents of variable Y  under normal noise:', pirca_inst.parents)

# We have a simple example with three variables for which we have each 1000 samples
n=1000
X=np.zeros((n,3))



X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.pareto(3,size=(n,1)) # THIS IS NOT NORMAL HERE
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 

    
# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.pareto(3,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 

data=[X_normal,X_faulty]

pirca_inst.compute_parents(data,targets=['Y'],test='permutation')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}

print('Estimated parents of variable Y  under heavy tailed noise:', pirca_inst.parents)

# We have a simple example with three variables for which we have each 1000 samples
n=1000
X=np.zeros((n,3))



# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))  # TARGET NOISE IS NOT HOMOGENOUS
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))

# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,2,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))

data=[X_normal,X_faulty]


pirca_inst.compute_parents(data,targets=['0','Y','1'],test='permutation')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}

print('Estimated parents of variable Y  under homogeneity violation:', pirca_inst.parents)

NameError: name 'LinearRegression' is not defined

### Next to the 'permutation' test, we have a different test best on confidence intervals, here indicated with 'bootstrap'. The main difference: all previous methods try to find plausible sets in an absolute sense (meaning plausible sets ensure noise homogeneity). The 'bootstrap' approach looks for 'the most' plausible sets, i.e. finding subsets that generate the most noise homogeneity. The main caveat: depending on the specific data distribution this can lead to wrong conlusions, let's see what happens in this example:

In [17]:
# We have a simple example with three variables for which we have each 1000 samples
np.random.seed(42)
n=1000
X=np.zeros((n,3))

model=LinearRegression()
pirca_inst=pirca(model=model)

# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))

# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,1,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))

data=[X_normal,X_faulty]

pirca_inst.compute_parents(data,targets=['Y'],test='bootstrap')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}
print('Estimated parents of variable Y  under normal noise:', pirca_inst.parents)

# We have a simple example with three variables for which we have each 1000 samples
n=1000
X=np.zeros((n,3))



X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.pareto(3,size=(n,1)) # THIS IS NOT NORMAL HERE
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 

    
# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.pareto(3,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1)) 

data=[X_normal,X_faulty]

pirca_inst.compute_parents(data,targets=['Y'],test='bootstrap')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}

print('Estimated parents of variable Y  under heavy tailed noise:', pirca_inst.parents)

# We have a simple example with three variables for which we have each 1000 samples
n=1000
X=np.zeros((n,3))



# We have the simple causal structure X0->Y->X1, which is in practice assumed to be unknown
# We first generate data where all variables act 'normally', and save this in a dictionary
X_normal={}
X_normal['0']=np.random.normal(0,1,size=(n,1))
X_normal['Y']=X_normal['0']+np.random.normal(0,1,size=(n,1))  # TARGET NOISE IS NOT HOMOGENOUS
X_normal['1']=X_normal['Y']+np.random.normal(0,2**(1/2),size=(n,1))

# We then introduce a 'fault' into the system by increasing the variance of X1, which affects X1 and X2
X_faulty={}
X_faulty['0']=np.random.normal(0,4,size=(n,1))
X_faulty['Y']=X_faulty['0']+np.random.normal(0,2,size=(n,1))
X_faulty['1']=X_faulty['Y']+np.random.normal(0,2**(1/2),size=(n,1))

data=[X_normal,X_faulty]


pirca_inst.compute_parents(data,targets=['0','Y','1'],test='bootstrap')

B=1000
plausibleS=[set(s) for s in pirca_inst.plausible['Y']]
if not not plausibleS:
    supphat=set.intersection(*plausibleS)
else:
    supphat={}

print('Estimated parents of variable Y  under homogeneity violation:', pirca_inst.parents)

Model Training Phase
|████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00%
 Computing Plausible Sets
|█████████████████████████---------------------------------------------------------------------------| 25.00%

C:\Users\amey\OneDrive - ASML\Documents\Github\Code_for_LICP\LICP\licp.py:135: UserWarning: No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.
  warnings.warn('No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.')


Estimated parents of variable Y  under normal noise: {'Y': {('0',): 0.99110422102479}}███████████████| 100.00%
Model Training Phase
|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180.00%
 Computing Plausible Sets
|██████████████████████████████████████████████████--------------------------------------------------| 50.00%

C:\Users\amey\OneDrive - ASML\Documents\Github\Code_for_LICP\LICP\licp.py:135: UserWarning: No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.
  warnings.warn('No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.')


Estimated parents of variable Y  under heavy tailed noise: {'Y': {('0',): 0.7627996227081058, ('1',): 0.5293688956639675}}
Model Training Phase
|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 126.67%
 Computing Plausible Sets
|████████--------------------------------------------------------------------------------------------| 8.33%

C:\Users\amey\OneDrive - ASML\Documents\Github\Code_for_LICP\LICP\licp.py:135: UserWarning: No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.
  warnings.warn('No timestamps provided. Calculating timestamps based on the assumptions that start and end time of all measurements are the same.')


Estimated parents of variable Y  under homogeneity violation: {'Y': {('0',): 0.2167594844867301, ('1',): 0.5755708956009704}, '0': {('1',): 0.1842458028027876}, '1': {('Y',): 0.8766156697221484}}


### Bootstrap does not work very well here, showing that we indeed need to adress the homogeneity condition